In [1]:
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

In [2]:
def load_images_from_folder(folders):
    class_images = []
    for folder in folders:
        images = []
        for filename in os.listdir(folder):
            img = Image.open(os.path.join(folder,filename))
            if img is not None:
                img = img.resize((299, 299))
                img_flip = ImageOps.flip(img)
                all_img = [img, img_flip]
                for i in all_img:
                    for r in [0, 90, 180, 270]:
                        images.append(np.array(i.rotate(r)))
        class_images.append(images)
    
    return class_images.copy()

In [3]:
def defined_input(class_images):
    features = []
    for images in class_images:
        for img in images:
            features.append(img)
    features = np.array(features)
    features = tf.keras.applications.inception_v3.preprocess_input(features, data_format=None)
    return features

In [4]:
#เตรียม output
def defined_output(class_images):
    label = []
    for count, images in enumerate(class_images):
        for i in range(len(images)):
            classes = len(class_images)
            output = np.zeros((classes,), dtype=int)
            output[count] = 1
            label.append(output)
    return np.array(label)

In [5]:
#load train_data
foldersname = []
for i in range(14):
    foldersname.append('./images/Head_Mid_Tail/class' + str(i) + '/train/tail') #path_to_folder, change file path this.

class_images = load_images_from_folder(foldersname)

X_train = np.copy(defined_input(class_images))
y_train = np.copy(defined_output(class_images))

In [6]:
#load validation_data
foldersname = []
for i in range(14):
    foldersname.append('./images/Head_Mid_Tail/class' + str(i) + '/validation/tail') #path_to_folder, change file path this.

class_images = load_images_from_folder(foldersname)

X_validation = np.copy(defined_input(class_images))
y_validation = np.copy(defined_output(class_images))

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout , GlobalAveragePooling2D

In [8]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(299,299,3))

base_model.trainable = False #Freeze Parameters

inputs = tf.keras.Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(14, activation='softmax')(x)

In [9]:
model = Model(inputs, predictions)

In [10]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)              

In [12]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_data = (X_validation, y_validation))

Epoch 1/10
57/57 [==============================] - 112s 2s/step - loss: 2.5256 - accuracy: 0.1718 - val_loss: 2.2375 - val_accuracy: 0.3750
Epoch 2/10
57/57 [==============================] - 113s 2s/step - loss: 2.1360 - accuracy: 0.3183 - val_loss: 1.9152 - val_accuracy: 0.5234
Epoch 3/10
57/57 [==============================] - 117s 2s/step - loss: 1.7579 - accuracy: 0.4455 - val_loss: 1.5796 - val_accuracy: 0.5859
Epoch 4/10
57/57 [==============================] - 117s 2s/step - loss: 1.4846 - accuracy: 0.5430 - val_loss: 1.3480 - val_accuracy: 0.6562
Epoch 5/10
57/57 [==============================] - 121s 2s/step - loss: 1.2539 - accuracy: 0.6184 - val_loss: 1.1877 - val_accuracy: 0.6562
Epoch 6/10
57/57 [==============================] - 117s 2s/step - loss: 1.0838 - accuracy: 0.6646 - val_loss: 1.0763 - val_accuracy: 0.6699
Epoch 7/10
57/57 [==============================] - 117s 2s/step - loss: 0.9599 - accuracy: 0.7032 - val_loss: 0.9959 - val_accuracy: 0.6777
Epoch 8/10
57

In [13]:
model.save('./model/head_mid_tail/tail_V2.h5') #path_to_save_model, change file path this.